# Spatial Raster Metadata: CRS, Resolution, and Extent

3 important spatial attributes associated with raster data: CRS, resolution, and spatial extent.

## 1. Coordinate Reference System

The CRS of a spatial object tells Python where the raster is located in geographic space. It also tells Python what math method should be used to "flatten" or prouject the raster in geographic space.

![projections](projections.jpg)

### What makes spatial data line up on a map?

It's important when working with spatial data in Python to identify the CRS applied to the data and retain it throughout data processing and analysis.

### View raster CRS

In [9]:
import os
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon, mapping
import rasterio as rio
from rasterio.mask import mask
from rasterio.plot import show

# Package created for the earth analytics program
import earthpy as et

# Get data and set working directory
et.data.get_data("colorado-flood")
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

# Define relative path to file
lidar_dem_path = os.path.join("data", "colorado-flood", "spatial", 
                              "boulder-leehill-rd", "pre-flood", "lidar",
                              "pre_DTM.tif")

# View crs of raster imported with rasterio
with rio.open(lidar_dem_path) as src:
    print(src.crs)


EPSG:32613


In [10]:
# Assign CRS to myCRS object
myCRS = src.crs

myCRS

CRS.from_epsg(32613)

The CRS EPSG code for your lidar_dem object is 32613. Next, you can look that EPSG code up on the spatial reference.org website to figure out what CRS it refers to and the associated units. In this case, you are using UTM zone 13 North.

You can view that the proj4string which tells us that the horizontal units of this project are in meters (m):

![UTM-zones](UTM-zones.png)

The CRS format, returned by python, is in a EPSG format. This means that the projection information is represented by a single number. However on the spatialreference.org website you can also view the proj4string which tell you a bit more about the horizontal units that the data are in. An overview of proj4 is below.

+proj=utm +zone=18 +datum=WGS84 +units=m +no_defs +ellps=WGS84 +towgs84=0,0,0

## Converting EPSG to Proj4

A package for this class called "earthpy" contains a dictionary that will help you convert EPSG codes into a Proj4string. This can be used with rasterio in order to determine the metadata for a given EPSG code. If you wish to know the units of the EPSG code above, you can do the following:

In [11]:
# Each key of the dictionary is an EPSG code
print(list(et.epsg.keys())[:10])

['29188', '26733', '24600', '32189', '4899', '29189', '26734', '7402', '26951', '29190']


In [12]:
# You can convert to proj4 like so:
proj4 = et.epsg["32613"]
print(proj4)

+proj=utm +zone=13 +datum=WGS84 +units=m +no_defs


In [14]:
# Finally you can convert this into a rasterio CRS like so:
crs_proj4 = rio.crs.CRS.from_string(proj4)
crs_proj4

CRS.from_epsg(32613)

In [15]:
# Finally you can convert this into a rasterio CRS like so:
crs_proj4 = rio.crs.CRS.from_string(proj4)
crs_proj4

CRS.from_epsg(32613)

You'll focus on the first few components of the CRS in this tutorial.
- +proj=utm The projection of the dataset. Your data are in Universal Transverse Mercator (UTM).
- +zone=18 The UTM projection divides up the world into zones, this element tells you which zone the data is in. Harvard Forest is in Zone 18.
- +datum=WGS84 The datum was used to define the center point of the projection. Your raster uses the WGS84 datum.
- +units=m This is the horizontal units that the data are in. Your units are meters.

Important: You are working with LIDAR data which has a Z or vertical value as well. While the horizontal units often match the vertical unbits of a raster they don't always! Be sure to check the metadata of your data to figure out the vertical units!

## Spatial extent

![extent](extent.png)

The spatial extent of a Python spatial object represents the geographic "edge". Extent represents the overall geographic coverage of the spatial object.

You can acess teh spatial extent using the .bounds attribute in rasterio.

In [16]:
src.bounds

BoundingBox(left=472000.0, bottom=4434000.0, right=476000.0, top=4436000.0)

## Raster resolution

A raster has horizontal (x and y) resolution. This resolution represents the area on the ground that each pixel covers. The units for your data are in meters as determined by the CRS above. In this case, your data resolution is 1x1. This means that each pixel represents a 1x1 m area on the ground. You can view the resolution of your data using the .res function.

In [17]:
# What is the x and y resolution for your raster data?
src.res

(1.0, 1.0)